In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Data/SlackData/lab-assignments

/content/drive/MyDrive/Data/SlackData/lab-assignments


#Importing Files

In [4]:
import os 
import glob
import numpy as np
import pandas as pd


In [ ]:
def normalize_reaction(dataframe):
  for i in range(len(dataframe.index)):
    series_object = dataframe.iloc[i]
    for name_index, eachname in enumerate(series_object["name"]):
      for eachuser in series_object["users"][name_index]:
        array_list.append([series_object["client_msg_id"], series_object["ts"], eachname, eachuser])

# File Path

client_msg_id = msg_id (unique)

user = user_id 

type = message

text = text

ts = when message is uploaded (unique)

reactions = reactions made for a thread

user_profile = profile info like name avatar photo etc extract this to take profile name

thread_ts = like parent ts if this thread is parent than they are same

Columns to be dropped: 

["team", "user_team", "source_team", "blocks", "is_locked", "subscribed", "last_read", "files", "upload", "edited", "attachments", "display_as_bot"]

Columns that are not required 
["reply_count", "reply_users_count", latest_reply", reply_users", "replies"] we don't need these because we will organize our frames accordingly. But I won't drop them immediately to check correctness. 

In [2]:
columns_to_be_dropped = ["team", "user_team", "source_team", "blocks", "is_locked", "subscribed", "last_read", "files", "upload", "edited", "attachments", "display_as_bot"]

Here I will create empty dataframes

In [5]:
user_info_df = pd.DataFrame(None, columns = ["user", "real_name"])
reactions_df = pd.DataFrame(None, columns = ["client_msg_id", "ts", "name", "users"])
thread_df = pd.DataFrame(None, columns = ["client_msg_id", "ts", "user", "text"])
thread_reactions_df = pd.DataFrame(None, columns = ["thread_ts", "parent_user_id", "ts", "user"])

In [ ]:
for filename in glob.glob('*.json'):
   with open(os.path.join(os.getcwd(), filename), 'r', encoding="utf8") as f: # open in readonly mode
    dataframe = pd.read_json(f)
    for column in columns_to_be_dropped:
      try:
        dataframe.drop(column, axis = 1, inplace = True)
      except:
        pass # Maybe I can made some log here like today there is no file uploaded.
    
    
    # Creating general thread dataframe
    thread_dataframe = dataframe.loc[:,["client_msg_id", "ts", "user", "text"]]
    thread_df = pd.concat([thread_df, thread_dataframe])

    # dataframe.ts.duplicated().any() returns false so every thread has different ts values which is normal because it is almost impossible to send at the same time to the nano seconds
    # Creating thread reactions dataframe
    try:
      thread_reactions_dataframe = dataframe.loc[:,["thread_ts", "parent_user_id", "ts", "user"]]
      thread_reactions_df = pd.concat([thread_reactions_df, thread_reactions_dataframe])
      thread_reactions_df.dropna(inplace=True)
    except:
      pass # No replies at this day
    try:
      # Creating new user profile dataframe
      user_profile_dataframe = pd.json_normalize(dataframe.user_profile)
      user_profile_dataframe.drop(["avatar_hash", "image_72", "first_name","display_name", "team", "name", "is_restricted", "is_ultra_restricted"], axis = 1, inplace = True)

      # Dropping user profile column and mergeimg them (most general dataframe)
      dataframe.drop("user_profile", axis = 1, inplace = True)
      dataframe = dataframe.merge(user_profile_dataframe, left_index = True, right_index = True)

      # After merging user_profile_dataframe and dataframe we may populate our user_info_df
      user_profile_dataframe = dataframe.loc[:,["user","real_name"]]
      user_profile_dataframe.drop_duplicates(inplace = True)
      user_profile_dataframe.dropna(inplace=True)
      user_info_df = pd.concat([user_info_df, user_profile_dataframe])
      user_info_df.drop_duplicates(inplace = True)

    except:
      pass
    # Creating individual reactions_dataframe 
    try:
      reactions_dataframe = dataframe[["client_msg_id", "ts", "reactions"]]
      reactions_dataframe.dropna(inplace = True)
      reactions_dataframe['name'] = reactions_dataframe['reactions'].explode().str['name'].groupby(level=0).agg(list)
      reactions_dataframe['users'] = reactions_dataframe['reactions'].explode().str['users'].groupby(level=0).agg(list) 
      array_list = [] # I am using this list in normalize reaction function 
      normalize_reaction(reactions_dataframe)
      reactions_dataframe = pd.DataFrame(array_list, columns = ["client_msg_id", "ts", "name", "users"])
      reactions_df = pd.concat([reactions_df, reactions_dataframe])
    except: # If it works no reaction at this day maybe we can make some logs here again
      pass
    
    


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
user_info_df
user_info_df.to_csv('user_info.csv', index = False)

In [ ]:
thread_df
thread_df.to_csv('thread.csv', index = False)

In [ ]:
thread_reactions_df
thread_reactions_df.to_csv("thread_reactions.csv", index = False)

In [ ]:
reactions_df
reactions_df.to_csv("reactions.csv", index = False)